In [0]:
# 00_config_auth — autenticação via SAS guardado no Secret Scope (job-safe)

# --- parâmetros (podem ser sobrescritos pelo Job via "Parameters") ---
# Se preferir, edite aqui os valores padrão.
try:
    dbutils.widgets.get               # garante que widgets existem
except:
    pass

def _wget(name, default):
    try:
        dbutils.widgets.text(name, default)
    except:
        # widget já existia
        pass
    return dbutils.widgets.get(name)

acc   = _wget("storage_account", "stgflowdev001")        # sem .dfs
scope = _wget("secret_scope",    "meu_scope")
key   = _wget("secret_key",      "sas-stgflowdev001")

# --- lê o SAS do Secret Scope ---
try:
    sas_raw = dbutils.secrets.get(scope, key)
except Exception as e:
    raise Exception(
        f"[Auth] Falha ao ler secret: scope='{scope}', key='{key}'. "
        "Verifique se o scope e a chave existem e se você tem permissão."
    ) from e

# --- limpa o SAS (remove quebras de linha/espaços e '?' inicial, se houver) ---
sas = sas_raw.strip().replace("\n", "").replace("\r", "")
if sas.startswith("?"):
    sas = sas[1:]

if not sas.startswith("sv="):
    raise ValueError(
        "[Auth] O valor do SAS não parece válido (não inicia com 'sv='). "
        "Confira o segredo salvo no scope."
    )

# --- configura o Spark para usar SAS ---
spark.conf.set(f"fs.azure.account.auth.type.{acc}.dfs.core.windows.net", "SAS")
spark.conf.set(f"fs.azure.sas.token.provider.type.{acc}.dfs.core.windows.net",
               "org.apache.hadoop.fs.azurebfs.sas.FixedSASTokenProvider")
spark.conf.set(f"fs.azure.sas.fixed.token.{acc}.dfs.core.windows.net", sas)

# --- caminhos base (para usar nas células seguintes) ---
bronze_base = f"abfss://bronze@{acc}.dfs.core.windows.net"
silver_base = f"abfss://silver@{acc}.dfs.core.windows.net"
gold_base   = f"abfss://gold@{acc}.dfs.core.windows.net"

print("[Auth] OK — SAS aplicado e caminhos definidos.")


In [0]:
%sql
-- Célula 1 (SQL)
CREATE OR REPLACE TEMP VIEW bronze_fx AS
SELECT *
FROM delta.`abfss://bronze@stgflowdev001.dfs.core.windows.net/fx_usdbrl/delta`;


In [0]:
%sql
-- Célula 2 (SQL)
SELECT base, symbol, rate, ref_date, provider, ingestion_ts
FROM bronze_fx
ORDER BY ingestion_ts DESC
LIMIT 20;


In [0]:
%sql
-- Célula 3 (SQL)
CREATE OR REPLACE TEMP VIEW silver_fx_stage AS
SELECT
  base,
  symbol,
  CAST(rate AS DOUBLE)          AS rate,
  TO_DATE(ref_date)             AS ref_date,
  COALESCE(provider,'unknown')  AS provider,
  ingestion_ts
FROM bronze_fx
WHERE symbol = 'BRL'
  AND base   = 'USD'
  AND ref_date IS NOT NULL
  AND rate IS NOT NULL
  AND CAST(rate AS DOUBLE) > 0;


In [0]:
%sql
-- Célula 4 (SQL)
CREATE OR REPLACE TEMP VIEW silver_fx_dedup AS
WITH ranked AS (
  SELECT *,
         ROW_NUMBER() OVER (PARTITION BY ref_date ORDER BY ingestion_ts DESC) AS rn
  FROM silver_fx_stage
)
SELECT base, symbol, rate, ref_date, provider, ingestion_ts
FROM ranked
WHERE rn = 1;


In [0]:
%sql
-- Célula 5 (SQL)
CREATE OR REPLACE TABLE delta.`abfss://silver@stgflowdev001.dfs.core.windows.net/fx_usdbrl/delta`
USING delta
PARTITIONED BY (year, month)
AS
SELECT
  ref_date,
  base,
  symbol,
  rate,
  provider,
  ingestion_ts,
  YEAR(ref_date)  AS year,
  MONTH(ref_date) AS month
FROM silver_fx_dedup;


In [0]:
%sql
-- Célula 6 (SQL)
SELECT ref_date, base, symbol, rate, provider, ingestion_ts
FROM delta.`abfss://silver@stgflowdev001.dfs.core.windows.net/fx_usdbrl/delta`
ORDER BY ref_date DESC;
